# Importing new entries from Zenodo communities
This notebook allows to import entries from Zenodo communities. It does not re-import entries that are already in our database.

In [1]:
import os
import requests
import bia_bob
import shutil
import pandas as pd

In [2]:
# workaround: Until our utilities are a python library, we need to copy it here.
#shutil.copy('../scripts/generate_link_lists.py', './generate_link_lists.py')

In [3]:
from generate_link_lists import load_dataframe
from generate_link_lists import update_yaml_file

In [4]:
token = os.getenv('ZENODO_API_KEY')
community = 'nfdi4bioimage'

response = requests.get('https://zenodo.org/api/records',
                        params={'communities': community,
                                'access_token': token})

## That's what's listed in the community

In [5]:
online_data = response.json()
hits = online_data["hits"]["hits"]

In [6]:
len(hits)

25

In [7]:
urls = [u["links"]["self_html"] for u in hits]
urls

['https://zenodo.org/records/14014252',
 'https://zenodo.org/records/14001388',
 'https://zenodo.org/records/13991322',
 'https://zenodo.org/records/13837146',
 'https://zenodo.org/records/13831274',
 'https://zenodo.org/records/11503289',
 'https://zenodo.org/records/13684187',
 'https://zenodo.org/records/13640979',
 'https://zenodo.org/records/13380289',
 'https://zenodo.org/records/13168693',
 'https://zenodo.org/records/12699637',
 'https://zenodo.org/records/11548617',
 'https://zenodo.org/records/11501662',
 'https://zenodo.org/records/11350689',
 'https://zenodo.org/records/11235513',
 'https://zenodo.org/records/11109616',
 'https://zenodo.org/records/11031747',
 'https://zenodo.org/records/10939520',
 'https://zenodo.org/records/10886750',
 'https://zenodo.org/records/10808486',
 'https://zenodo.org/records/10793700',
 'https://zenodo.org/records/10730424',
 'https://zenodo.org/records/10687659',
 'https://zenodo.org/records/10617006',
 'https://zenodo.org/records/10389955']

## Checking what we already have

In [8]:
df = load_dataframe("../resources/")

all_urls = str(df["url"].tolist())
#all_urls

Adding nfdi4bioimage.yml


## Identifying entries we are missing yet

In [9]:
new_urls = []
for url in urls:
    if url not in all_urls:
        new_urls.append(url)

new_urls

['https://zenodo.org/records/13837146',
 'https://zenodo.org/records/11503289',
 'https://zenodo.org/records/13684187',
 'https://zenodo.org/records/13640979',
 'https://zenodo.org/records/13380289',
 'https://zenodo.org/records/12699637',
 'https://zenodo.org/records/11501662',
 'https://zenodo.org/records/11350689',
 'https://zenodo.org/records/11235513',
 'https://zenodo.org/records/11031747',
 'https://zenodo.org/records/10939520',
 'https://zenodo.org/records/10886750',
 'https://zenodo.org/records/10808486',
 'https://zenodo.org/records/10793700',
 'https://zenodo.org/records/10730424',
 'https://zenodo.org/records/10687659',
 'https://zenodo.org/records/10617006',
 'https://zenodo.org/records/10389955']

## Saving new entries

In [10]:
with open('../resources/nfdi4bioimage.yml', 'a') as file:
    file.write("\n")
    for url in new_urls:
        file.write("- url: " + url + '\n')

In [11]:
update_yaml_file("../resources/nfdi4bioimage.yml")

URL: https://focalplane.biologists.com/2023/07/26/sharing-your-poster-on-figshare/
URL: https://biapol.github.io/blog/marcelo_zoccoler/omero_scripts/readme.html
URL: https://biapol.github.io/blog/robert_haase/browsing_idr/readme.html
URL: https://biapol.github.io/blog/mara_lampert/getting_started_with_mambaforge_and_python/readme.html
URL: https://datamanagement.hms.harvard.edu/news/promoting-data-management-nikon-imaging-center-and-cell-biology-microscopy-facility
URL: https://blog.delmic.com/data-handling-in-large-scale-electron-microscopy
URL: https://focalplane.biologists.com/2023/06/01/tracking-in-napari/
URL: https://focalplane.biologists.com/2023/05/03/feature-extraction-in-napari/
URL: https://focalplane.biologists.com/2023/03/30/annotating-3d-images-in-napari/
URL: https://focalplane.biologists.com/2022/12/08/managing-scientific-python-environments-using-conda-mamba-and-friends/
URL: https://focalplane.biologists.com/2023/04/13/quality-assurance-of-segmentation-results/
URL: h